In [1]:
import pandas as pd
import pickle
import numpy as np

In [3]:
df=pd.read_csv('/home/wapun/Documents/demand/arima/csv/Load&TD.csv')
df['Time']=pd.to_datetime(df.Time,utc=True)
df.set_index('Time',inplace=True,drop=True)
df.index=df.index.tz_convert('Europe/Warsaw')

In [4]:
df

,PSEforecast,Load,TD,TDdet,TG,TGdet,ARIMALOAD,ARIMATD1
Time,,,,,,,,
2015-01-10 00:00:00+01:00,17000.0,17109.375,-2279.250,19388.625,-48.125,17157.500,NaN,NaN
2015-01-10 01:00:00+01:00,16200.0,16438.150,-2234.775,18672.925,-46.600,16484.750,NaN,NaN
2015-01-10 02:00:00+01:00,15800.0,16122.300,-2148.525,18270.825,-8.800,16131.100,NaN,NaN
2015-01-10 03:00:00+01:00,15700.0,15987.900,-2082.875,18070.775,26.000,15961.900,NaN,NaN
2015-01-10 04:00:00+01:00,15700.0,15951.900,-2266.525,18218.425,-220.775,16172.675,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-02-28 17:00:00+01:00,23557.0,23578.163,576.713,23001.450,-110.000,23688.163,20084.256328,-156.113809
2024-02-28 18:00:00+01:00,23558.0,23746.113,507.601,23238.512,242.325,23503.788,20762.788245,-159.858698
2024-02-28 19:00:00+01:00,23408.0,23638.600,518.563,23120.037,95.812,23542.788,21439.673164,-161.895222


In [6]:
with open('models/TD.pkl', 'rb') as pkl:
    TD = pickle.load(pkl)
with open('models/TG.pkl', 'rb') as pkl:
    TG = pickle.load(pkl)
with open('models/Load.pkl', 'rb') as pkl:
    Load = pickle.load(pkl)
TD


ARIMA(order=(6, 0, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [33]:

def dataSplit(dataframe,end_train):
    day=pd.to_timedelta(1,'day')
    train_data=df.loc[(dataframe.index<end_train)]
    test_data=df.loc[(dataframe.index>=end_train)&(dataframe.index.date<(end_train+2*day).date())]
    
    return train_data,test_data


In [34]:
TG.predict(4)

2022-02-28 11:00:00+01:00   -205.259200
2022-02-28 12:00:00+01:00    -47.308455
2022-02-28 13:00:00+01:00     -5.423784
2022-02-28 14:00:00+01:00    -15.609709
Freq: H, dtype: float64

In [35]:
end_train=pd.to_datetime('2022-02-28 11:00',utc=False)
end_train=end_train.tz_localize('Europe/Warsaw')
day=pd.to_timedelta(1,'day')

In [36]:

def pred36(dataframe,end_train,model):
    
    trainData,testData=dataSplit(dataframe,end_train)
    pred=model.predict(len(testData))
    model.update(testData.loc[testData.index.date>end_train.date(),[name for name, obj in globals().items() if obj is model][0]])
    return pred[-len(testData.loc[testData.index.date>end_train.date()]):]

In [8]:
for date in pd.date_range(start=end_train,end=df.tail(1).index[0]):
    df.loc[df.index.date==(date+day).date(),'ARIMALOAD']=pred36(df,date,Load)

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [18]:
end_train=pd.to_datetime('2022-02-28 11:00',utc=False)
end_train=end_train.tz_localize('Europe/Warsaw')
for date in pd.date_range(start=end_train,end=df.tail(1).index[0]):
    df.loc[df.index.date==(date+day).date(),'ARIMATD1']=pred36(df,date,TD)

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [37]:
end_train=pd.to_datetime('2022-02-28 11:00',utc=False)
end_train=end_train.tz_localize('Europe/Warsaw')
for date in pd.date_range(start=end_train,end=df.tail(1).index[0]):
    df.loc[df.index.date==(date+day).date(),'ARIMATG1']=pred36(df,date,TG)

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [38]:
df

,PSEforecast,Load,TD,TDdet,TG,TGdet,ARIMALOAD,ARIMATD1,ARIMATG1
Time,,,,,,,,,
2015-01-10 00:00:00+01:00,17000.0,17109.375,-2279.250,19388.625,-48.125,17157.500,NaN,NaN,NaN
2015-01-10 01:00:00+01:00,16200.0,16438.150,-2234.775,18672.925,-46.600,16484.750,NaN,NaN,NaN
2015-01-10 02:00:00+01:00,15800.0,16122.300,-2148.525,18270.825,-8.800,16131.100,NaN,NaN,NaN
2015-01-10 03:00:00+01:00,15700.0,15987.900,-2082.875,18070.775,26.000,15961.900,NaN,NaN,NaN
2015-01-10 04:00:00+01:00,15700.0,15951.900,-2266.525,18218.425,-220.775,16172.675,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2024-02-28 17:00:00+01:00,23557.0,23578.163,576.713,23001.450,-110.000,23688.163,20084.256328,-156.113809,6.900836
2024-02-28 18:00:00+01:00,23558.0,23746.113,507.601,23238.512,242.325,23503.788,20762.788245,-159.858698,-24.095153
2024-02-28 19:00:00+01:00,23408.0,23638.600,518.563,23120.037,95.812,23542.788,21439.673164,-161.895222,-5.981781


create small df based on df  
predict add predicted values  
make second prediction  


In [23]:
df.to_csv('Load&TD&TG.csv')

dalej jakis problem z długością jest